In [9]:
import math
import scipy
import numpy

def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    # divisor = (volitility * mature_time * math.sqrt(mature_time))
    # d1 = ((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (divisor))
    # d2 = (((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (divisor)) - divisor)
    call = (spot_price * scipy.stats.norm.cdf(((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))))) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf((((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))) - (volitility * mature_time * math.sqrt(mature_time))))
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -(((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))) - (volitility * mature_time * math.sqrt(mature_time)))) - spot_price * scipy.stats.norm.cdf(-((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))))
    if callput == "put":
        return strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
            -(((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))) - (volitility * mature_time * math.sqrt(mature_time)))) - spot_price * scipy.stats.norm.cdf(-((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))))
    return (spot_price * scipy.stats.norm.cdf(((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))))) - strike_price * math.exp(
            -risk_rate * mature_time) * scipy.stats.norm.cdf((((math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / ((volitility * mature_time * math.sqrt(mature_time)))) - (volitility * mature_time * math.sqrt(mature_time))))

    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))


print(black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))
print(black_scholes(100, 96, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 96, 0.05, 0.5, 0.2, "put"))

0.03311723312069925
17.070306676520616
7.009859140996113
0.6396106957160494


In [2]:
import math

import numpy

def binomial_tree(stock_price, strike_price, interest_rate, volatility, time, steps, option_type, exercise_type):
    volatility = volatility * time
    stock_tree = [[stock_price]]
    for step in range(1, steps + 1):
        price_row = []
        for item in range(0, step + 1):
            if item == step:
                price = stock_tree[step - 1][step - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            elif item == 0:
                price = stock_tree[step - 1][0] * (1 / (math.exp(volatility * numpy.sqrt((time / steps)))))
            else:
                price = stock_tree[step - 1][item - 1] * (math.exp(volatility * numpy.sqrt((time / steps))))
            price_row.append(price)
        stock_tree.append(price_row)
    tree = [[-1]]
    for step in range(1, steps):
        child = []
        for item in range(step + 1):
            child.append(-1)
        tree.append(child)
    level = []
    for step in range(steps + 1):
        price = stock_tree[steps][step]
        if option_type == 'call':
            if price <= strike_price:
                value = 0
            else:
                value = price - strike_price
        else:
            if strike_price > price:
                value = strike_price - price
            else:
                value = 0

        level.append(value)
    tree.append(level)

    for step in range(steps - 1, -1, -1):
        col = tree[step + 1]
        for item in range(step + 1):
            if option_type == 'put':
                if exercise_type == 'american':
                    if value < strike_price - stock_tree[step][item]:
                        american_value = strike_price - stock_tree[step][item]
                    else:
                        american_value = value

                    value = (math.exp(-interest_rate * (time / steps))) * ((math.exp(
                        interest_rate * (time / steps)) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                        volatility * numpy.sqrt((time / steps)))) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps)))))) * col[item + 1] + (1 - (
                            math.exp(interest_rate * (time / steps)) - (
                            1 / (math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                        volatility * numpy.sqrt((time / steps)))) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps))))))) * col[item])
                    if american_value < value:
                        tree[step][item] = value
                    else:
                        tree[step][item] = american_value

                else:
                    tree[step][item] = (math.exp(-interest_rate * (time / steps))) * ((math.exp(
                        interest_rate * (time / steps)) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                        volatility * numpy.sqrt((time / steps)))) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps)))))) * col[item + 1] + (1 - (
                            math.exp(interest_rate * (time / steps)) - (
                            1 / (math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                        volatility * numpy.sqrt((time / steps)))) - (1 / (
                        math.exp(volatility * numpy.sqrt((time / steps))))))) * col[item])
            else:
                tree[step][item] = (math.exp(-interest_rate * (time / steps))) * ((math.exp(
                    interest_rate * (time / steps)) - (1 / (
                    math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                    volatility * numpy.sqrt((time / steps)))) - (1 / (
                    math.exp(volatility * numpy.sqrt((time / steps)))))) * col[item + 1] + (1 - (
                        math.exp(interest_rate * (time / steps)) - (
                        1 / (math.exp(volatility * numpy.sqrt((time / steps)))))) / ((math.exp(
                    volatility * numpy.sqrt((time / steps)))) - (1 / (
                    math.exp(volatility * numpy.sqrt((time / steps))))))) * col[item])
    tree = tree[0][0]
    return tree



print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'european'))
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'european'))
print(binomial_tree(100, 95, 0.05, 0.20, 0.5, 180, 'call', 'european'))
print(binomial_tree(100, 95, 0.05, 0.20, 0.5, 180, 'put', 'european'))
print('--')
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print(binomial_tree(100, 120, 0.05, 0.20, 0.5, 180, 'put', 'american'))
print(binomial_tree(100, 95, 0.05, 0.20, 0.5, 180, 'call', 'american'))
print(binomial_tree(100, 95, 0.05, 0.20, 0.5, 180, 'put', 'american'))


0.03396405508481631
17.071153498486204
7.831833879451344
0.48627552214449243
--
0.03396405508481631
79.10639944187882
7.831833879451344
54.7111117206877


In [3]:
import scipy


def monte_carlo(stock_price, strike_price, time, interest_rate, volatility, type, steps, simulations):
    volatility *= time
    st = stock_price * numpy.exp(numpy.cumsum(((interest_rate - 0.5 * volatility * volatility) * time / steps) + (
            volatility * numpy.sqrt(time / steps)) * numpy.random.standard_normal(size=(steps, simulations)), axis=0))
    if type == "put":
        values = numpy.maximum(strike_price - st[-1], 0)
    else:
        values = numpy.maximum(st[-1] - strike_price, 0)
    val = 0
    for i in range(0, len(values)):
        val = values[i] + val
    return val / len(values) * math.exp(-interest_rate * time)


print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "put", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "put", 500, 200000))

0.03357200489445483
17.049408418051616
7.004069589102746
0.6544535203253329
